In [2]:
import plotly.graph_objects as go
import pandas as pd

# reading input log file
import sys
from csv import excel

path='../save_outputs/vol-vfd'
task_cnt = 8

# fsim = f'{path}/hm-sim.f100.n{task_cnt}.r100.log.xlsx'
# fagg = f'{path}/hm-agg.f100.n{task_cnt}.r100.log.xlsx'

fvolsim=f'{path}/prov-vfd-sim-vol.csv'
fvfdsim=f'{path}/prov-vfd-sim-vfd.csv'

fvolsim=f'{path}/prov-vfd-agg-vol.csv'
fvfdsim=f'{path}/prov-vfd-agg-vfd.csv'

In [3]:
def humansize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1000 and i < len(suffixes)-1:
        nbytes /= 1000.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [4]:
class SankeyData:
    
    # From Tazer
    df_stat = list()
    # Sankey in Plotly, node - vertex, link - edge
    nodes = pd.DataFrame(columns=['label', 'color'])
    links = pd.DataFrame(columns=['source', 'target', 'value', 'label'])
    
    color_map = {"task": "#ab2626", # read
                 "vol-element": "#26ab56", #green
                 "vfd-address": "#1e81b0", #blue
                 "file": "#154c79", # dark blue
                 "read": "#bf7777", # light read
                 "write": "#b277bf", #light purpul
                 "none": "grey",
                }
    block_size = 1024 # default block_size
    output_suffix = "hermesvfd-example"
        
    def __init__(self):
        pass
        
    def load_stat(self, df_stat, params):
        
        self.df_stat.append({ "df": df_stat,
                              "source": params['source'],
                              "label": params['label'],
                              "size": params['size'],
                             "target": params['target']
                            })

        # add a node entry (task)
        self.set_taskname(params['source'])

        # add a node entry (input/output file)
        # one filename entry is allowed
        # if len(self.nodes[self.nodes.label == params['source']]) == 0:
        #     self.add_node({'label': params['source'],
        #                    'color': self.color_map['file']})
        #     '''
        #     self.add_node({'label': 'no read',
        #               'color': self.color_map['none']})
        #     self.add_node({'label': 'no write',
        #               'color': self.color_map['none']})
        #     '''
        
    def set_taskname(self, name):
        if len(self.nodes[self.nodes.label == name]) == 0:
            self.add_node({'label': name,
                           'color': self.color_map['task']})
            
    def set_output_suffix(self,suffix):
        self.output_suffix = suffix
        print(f"Output Name : sankey-diagram-plotly-{self.output_suffix}.html")
                    
    def get_node(self, idx):
        return self.nodes[idx]
    
    def add_node(self, node_dict):
        # label, color, customdata, x, y
        x = pd.DataFrame([node_dict])
        self.nodes = pd.concat([self.nodes, x], ignore_index=True)
        new_id = len(self.nodes)
        
    def get_link(self, name):
        return self.link[name]
    
    def set_links(self):
        pass
    
    def set_block_size(self,size):
        self.block_size =  size

    def build_links(self, key, no_rw=False):
        # must pass a key
        links = []
        for v in self.df_stat:
            # input (r)
            cnt = v['df'][key].nunique()
            access_cnt = len(v['df'][key])
            label = v['label']
            source = v['source']
            target = v['target']
            s = int(self.nodes[self.nodes.label == source].index.values)
            t = int(self.nodes[self.nodes.label == target].index.values)
            
            s2t = {
                'source': s,
                'target': t,
                'label': label,
                'value': access_cnt
            }

            links.append(s2t)

        links = pd.DataFrame(links)
        self.links = links
    
    def plot(self):
        n = self.nodes[['label','color']].to_dict('list')
        l = self.links.to_dict('list')
        fig = go.Figure(data=[go.Sankey(
            node = n,
            link = l)])
        fig.show()
        fig.write_html(f"{path}/sankey-vol-vfd-{self.output_suffix}.html")

    def reset(self):
        del(self.df_stat)
        del(self.nodes)
        del(self.links)

In [ ]:
# simulation graph, build 2 dataframes
# # sim -> io_access_idx -> logical_addr -> residule_100.5 
# fsim_vol='../save_outputs/vol-vfd/prov-vfd-sim-vol.parquet'
# df_sim_vol = pd.read_parquet(fsim_vol)
# # print(df_sim_vol.head(3))

# df_sim2vol = df_sim_vol[['io_access_idx', 'logical_addr', 'access_size']].copy()
# # hardcode logical_addr=0 to 17400
# df_sim2vol['logical_addr'] = df_sim2vol['logical_addr'].replace(0,17400)
# print(df_sim2vol.head(3))

# sd = SankeyData()

# # new_df = fsim_vol[['io_access_idx', 'logical_addr', 'file_name']].copy()
# # new_df.rename(columns={"A": "a", "B": "c"})
# sd.load_stat(df_sim2vol, {'source': 'sim_emulator.py',
#                             'label': 'read',
#                             'size':df_sim2vol['access_size'].sum(),
#                             'target': df_sim2vol['io_access_idx']})

# sd.load_stat(df_sim2vol, {'source': df_sim2vol['io_access_idx'],
#                             'label': 'read',
#                             'size':df_sim2vol['access_size'].sum(),
#                             'target': df_sim2vol['logical_addr']})

# sd.build_links(key='io_access_idx')
# sd.set_output_suffix("sim")
# sd.plot()

# fsim_vfd='../save_outputs/vol-vfd/prov-vfd-sim-vfd.csv'
# df_vol2res = fsim_vfd[['logical_addr', 'logical_addr', 'file_name', 'access_size']].copy()

# sd.load_stat(fsim_vfd, {'source': fsim_vfd['logical_addr'],
#                             'label': 'read',
#                             'size':df_sim2vol['access_size'].sum(),
#                             'target': 'residue_100_output.h5'})

# sd.build_links(key='logical_addr')

# sd.set_output_suffix("sim")

# sd.plot()

# sd.nodes
# sd.links

In [26]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    color_map = {"task": "#ab2626", # read
                 "io_access_idx": "#26ab56", #green
                 "logical_addr": "#1e81b0", #blue (lighter)
                 "file": "#154c79", # dark blue
                 "read": "#bf7777", # light read
                 "write": "#b277bf", #light purpul
                 "none": "grey",
                }
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig
  
  
# fig = genSankey(df_merged,cat_cols=['io_access_idx_x','logical_addr','io_access_idx_y','file_name'],value_cols='count',title='sim_emulator')
# import plotly
# import plotly.plotly as py
# plotly.offline.plot(fig, validate=False)

In [6]:
fsim_vol='../save_outputs/vol-vfd/prov-vfd-sim-vol.parquet'
fsim_vfd='../save_outputs/vol-vfd/prov-vfd-sim-vfd.parquet'

df_sim_vol = pd.read_parquet(fsim_vol)
df_sim_vfd = pd.read_parquet(fsim_vfd)
# print(df_sim_vol.head(3))

df_sim2vol = df_sim_vol[['io_access_idx', 'logical_addr', 'access_size']].copy()
# hard code
df_sim2vol['logical_addr'] = df_sim2vol['logical_addr'].replace(0,17400)
# df_sim2vol.rename(index={"logical_addr": "logical_addr_vfd",})

df_vol2res = df_sim_vfd[['logical_addr', 'file_name', 'access_size', 'io_access_idx']].copy()
# df_vol2res.rename(columns={'io_access_idx': 'io_access_idx_vfd'})
df_merged = df_sim2vol.merge(df_vol2res, how='inner', on=['logical_addr'])

# df1['value']=df1[['time_by_hour','id']].merge(df2,how='left').value
# df1.merge(df2, left_on='lkey', right_on='rkey')

# populate file_name
# task0000/residue_100.h5
df_merged['file_name'] = df_merged['file_name'].fillna('task0000/residue_100.h5')

print(df_merged.columns)
print(df_merged.head(5))


Index(['io_access_idx_x', 'logical_addr', 'access_size_x', 'file_name',
       'access_size_y', 'io_access_idx_y'],
      dtype='object')
   io_access_idx_x  logical_addr  access_size_x                file_name  \
0                1       17400.0           9260  task0000/residue_100.h5   
1                2       17400.0           9344  task0000/residue_100.h5   
2                3       17400.0           9272  task0000/residue_100.h5   
3                4       17400.0           9328  task0000/residue_100.h5   
4                5       54712.0           9396  task0000/residue_100.h5   

   access_size_y  io_access_idx_y  
0          37312                7  
1          37312                7  
2          37312                7  
3          37312                7  
4          37728                8  
